In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Modifying the path so we can import from src directory.
import sys
import os
sys.path.append(os.path.abspath('..'))

from collections import Counter, defaultdict
from itertools import chain
import copy
import pickle
import random
import time

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
from pyvis.network import Network

from src.example_graphs import simple_undirected_graph, simple_directed_graph
from src.UndirectedGraph import UndirectedGraph
from src.DirectedGraph import DirectedGraph
from src.DataLoader import DataLoader
from src.GraphCreator import GraphCreator, NetworkXGraphCreator

from src.io_helpers import pickle_obj, load_pickled_obj
from src.networkx_helpers import combine_graphs
from src.networkx_multigraph_helpers import (get_edge_attrs, aggregate_numeric_properties,
                                             sum_numeric_properties, count_edges)

In [3]:
ROOT_DIRECTORY = os.path.split(os.getcwd())[0]
DATA_DIRECTORY = os.path.join(ROOT_DIRECTORY, 'data')
PICKLED_DATA_DIRECTORY = os.path.join(ROOT_DIRECTORY, 'data_pickle')

## Load our data

In [4]:
G_weighted = load_pickled_obj(os.path.join(PICKLED_DATA_DIRECTORY, 'networkx_weighted_full.pickle'))

# If you Went on a Random Walk...Where Would You End Up?
https://stackoverflow.com/questions/37311651/get-node-list-from-random-walk-in-networkx (A resource that I haven't read yet. You can probably do this using matrix multiplication.)

## Let's start with a subset of the graph again

In [5]:
G_weighted_subset = G_weighted.edge_subgraph(G_weighted.edges('vegan'))

In [6]:
def random_node(graph):
    nodes = list(graph.nodes)
    
    return random.choice(nodes)

In [ ]:
class RandomWalk:
    
    def __init__(self, graph, current_node=None, silent=True):
        self.steps_taken = 0
        self.blocked = False
        self.graph = graph
        if current_node:
            self.current_node = current_node
        else:
            self.current_node = random_node(self.graph)
        self.nodes_seen = defaultdict(int)
        self.nodes_seen[self.current_node] += 1
        self.silent = silent
    
    def walk(self):
        if self.blocked:
            if not self.silent:
                print("Can't take any more steps.")
        else:
            neighbor_nodes = []
            for neighbor in self.graph.adj[self.current_node]:
                neighbor_weight = self.graph.adj[self.current_node][neighbor]['weight']
                neighbor_nodes.extend([neighbor] * neighbor_weight)
            if neighbor_nodes:
                next_node = random.choice(neighbor_nodes)
                self.current_node = next_node
                self.nodes_seen[self.current_node] += 1
                self.steps_taken += 1
            else:
                self.blocked = True
                if not self.silent:
                    print("Can't take any more steps.")
    
    def delete_graph(self):
        """Method to delete the graph so we aren't using a bunch of memory."""
        self.graph = None

## Increasing Efficiency of RandomWalk class
Our RandomWalk walk was slowing down significantly, and I discovered that it was because we were storing a full copy of the graph inside of every instance of the RandomWalk class. With the graph size we were dealing with, this added up very quickly and probably locked down all the computer's memory.

In [7]:
from src.os_helpers import getsize

ModuleNotFoundError: No module named 'src.os_helpers'

In [ ]:
walk1 = RandomWalk(G_weighted)

In [ ]:
for i in range(100):
    walk1.walk()
    if walk1.blocked:
        break

In [ ]:
print(walk1.steps_taken)
print(Counter(walk1.nodes_seen).most_common())

In [ ]:
print(getsize(walk1))

In [ ]:
walk1.delete_graph()

In [ ]:
getsize(walk1)

In [ ]:
walk2 = RandomWalk(G_weighted)
for i in range(100):
    walk2.walk()
    if walk2.blocked:
        break

In [ ]:
print(walk2.steps_taken)
print(Counter(walk2.nodes_seen).most_common())

In [ ]:
getsize(walk2)

In [ ]:
walk2.delete_graph()

In [ ]:
getsize(walk2)

In [ ]:
random_walks = [walk1, walk2]

In [ ]:
getsize(random_walks)

## Go on the walks! You can change the i-range here (number of trials) and j-range (max length of the walk before we shut it down)

In [ ]:
for i in range(10000):
    walk = RandomWalk(G_weighted)
    if i % 100 == 0:
        print(i)
    for j in range(1000):
        walk.walk()
        if walk.blocked:
            break
    walk.delete_graph()
    random_walks.append(copy.deepcopy(walk))

## Graphing the distribution of steps taken

In [ ]:
# This trial had max of 1000 steps per walk, and 10,000 trials.
step_counts = [walk.steps_taken for walk in random_walks]
fig = plt.figure(figsize=(12, 6))
ax = sns.distplot(step_counts)
_ = ax.set_title("Histogram of Steps Taken During Random Walks - Max-Steps of 1,000 w/ 10,000 trials")
_ = ax.set_ylabel("% of data")
_ = ax.set_xlabel("# of steps taken")

In [ ]:
# These are the step counts from a previous trial.
# Max of 100 steps per walk, with 1000 trials.
step_counts_100_1000 = [17, 29, 1, 6, 8, 15, 14, 6, 0, 55, 4, 18, 9, 77, 33, 27, 0, 19, 41, 31, 6, 23, 25, 14, 2, 0, 34, 22, 11, 24, 21, 0, 94, 25, 28, 22, 11, 61, 0, 44, 0, 11, 40, 9, 20, 1, 43, 18, 6, 35, 0, 39, 1, 40, 16, 4, 5, 28, 14, 100, 17, 2, 5, 12, 15, 37, 34, 12, 23, 31, 0, 0, 61, 34, 21, 0, 18, 17, 4, 41, 25, 0, 48, 33, 0, 28, 9, 3, 5, 2, 0, 12, 47, 33, 41, 21, 20, 0, 15, 0, 2, 2, 5, 2, 41, 7, 20, 44, 10, 10, 35, 16, 42, 17, 1, 19, 0, 3, 54, 0, 2, 22, 9, 0, 24, 10, 40, 13, 40, 17, 24, 29, 0, 0, 27, 21, 16, 7, 7, 1, 20, 0, 1, 3, 0, 42, 16, 1, 43, 12, 2, 0, 10, 27, 0, 2, 0, 1, 65, 0, 3, 6, 1, 3, 19, 19, 12, 3, 3, 8, 7, 4, 7, 24, 1, 40, 0, 3, 6, 1, 26, 2, 56, 8, 5, 15, 100, 16, 33, 10, 11, 34, 3, 2, 1, 19, 22, 0, 2, 37, 1, 0, 21, 12, 0, 1, 15, 0, 0, 0, 0, 53, 13, 100, 9, 13, 40, 46, 17, 13, 2, 29, 16, 4, 1, 22, 26, 6, 26, 13, 3, 2, 41, 0, 27, 4, 16, 0, 33, 11, 26, 40, 43, 6, 42, 2, 2, 19, 8, 11, 14, 0, 0, 0, 11, 11, 0, 13, 22, 0, 12, 10, 7, 1, 18, 0, 2, 1, 7, 22, 2, 61, 39, 6, 45, 5, 5, 0, 7, 9, 23, 9, 4, 4, 25, 69, 0, 1, 14, 4, 5, 4, 4, 0, 22, 13, 0, 45, 9, 22, 1, 13, 68, 1, 6, 20, 0, 12, 24, 0, 30, 2, 18, 1, 7, 57, 0, 10, 15, 0, 7, 27, 8, 0, 20, 5, 5, 6, 10, 21, 16, 13, 12, 21, 1, 0, 2, 6, 25, 10, 8, 19, 0, 83, 28, 13, 5, 0, 3, 0, 40, 27, 63, 1, 11, 8, 22, 23, 1, 4, 2, 0, 0, 19, 1, 3, 24, 19, 0, 10, 9, 10, 5, 3, 18, 16, 1, 0, 22, 0, 74, 8, 21, 45, 40, 0, 55, 16, 28, 6, 62, 10, 12, 9, 4, 6, 6, 0, 15, 15, 0, 39, 1, 100, 43, 10, 36, 0, 18, 12, 3, 27, 57, 3, 0, 4, 2, 28, 24, 49, 2, 0, 15, 19, 34, 7, 0, 10, 40, 8, 20, 1, 19, 19, 17, 0, 57, 8, 19, 22, 30, 0, 10, 2, 2, 7, 5, 26, 6, 58, 23, 3, 16, 0, 3, 46, 3, 40, 3, 6, 31, 33, 52, 4, 5, 13, 0, 20, 5, 13, 22, 23, 0, 46, 73, 28, 1, 53, 10, 11, 10, 12, 13, 100, 21, 61, 0, 0, 0, 0, 8, 24, 1, 15, 16, 22, 6, 14, 8, 6, 3, 44, 15, 7, 25, 0, 9, 5, 0, 7, 15, 3, 22, 6, 16, 36, 6, 26, 12, 55, 0, 29, 5, 23, 2, 4, 8, 51, 84, 31, 1, 21, 17, 60, 52, 46, 5, 10, 0, 62, 24, 38, 13, 100, 4, 53, 16, 33, 9, 10, 29, 0, 3, 24, 64, 10, 4, 18, 6, 0, 11, 0, 46, 4, 36, 0, 3, 2, 6, 17, 3, 0, 23, 5, 3, 0, 25, 61, 9, 19, 8, 1, 17, 24, 1, 0, 10, 0, 0, 30, 45, 12, 2, 38, 15, 0, 0, 18, 46, 0, 0, 27, 32, 35, 9, 0, 11, 46, 6, 79, 5, 3, 59, 17, 73, 19, 2, 25, 13, 47, 4, 21, 20, 0, 14, 4, 100, 75, 0, 0, 63, 89, 25, 12, 15, 22, 15, 2, 14, 0, 55, 40, 42, 1, 64, 3, 0, 13, 38, 5, 0, 4, 0, 1, 0, 3, 3, 0, 10, 0, 22, 39, 7, 9, 6, 79, 7, 16, 9, 0, 7, 54, 0, 6, 3, 15, 17, 37, 10, 1, 12, 45, 100, 0, 34, 6, 7, 13, 28, 0, 19, 65, 59, 0, 2, 28, 35, 18, 45, 26, 2, 1, 2, 15, 7, 6, 10, 3, 17, 0, 45, 0, 16, 1, 21, 49, 77, 13, 27, 25, 11, 0, 42, 0, 9, 0, 1, 0, 60, 4, 61, 19, 17, 6, 9, 23, 5, 0, 2, 100, 29, 5, 0, 2, 13, 14, 31, 44, 12, 24, 12, 66, 51, 10, 68, 7, 2, 1, 9, 9, 55, 9, 17, 9, 0, 7, 3, 15, 13, 5, 12, 0, 0, 12, 0, 26, 2, 2, 0, 0, 21, 4, 0, 12, 97, 5, 58, 0, 1, 37, 21, 39, 28, 43, 42, 13, 0, 5, 6, 63, 0, 26, 0, 7, 10, 0, 17, 23, 13, 1, 19, 0, 23, 22, 2, 2, 3, 11, 13, 8, 33, 38, 1, 22, 12, 42, 0, 11, 0, 0, 7, 0, 12, 1, 0, 1, 10, 84, 43, 12, 9, 14, 95, 14, 30, 4, 43, 3, 24, 16, 15, 0, 7, 6, 10, 25, 2, 71, 6, 1, 13, 59, 7, 50, 51, 32, 0, 0, 17, 1, 0, 1, 30, 3, 1, 34, 8, 24, 31, 46, 0, 23, 33, 1, 1, 3, 52, 9, 2, 43, 21, 32, 4, 51, 2, 22, 53, 11, 18, 59, 48, 24, 10, 2, 22, 23, 8, 23, 7, 55, 0, 4, 36, 16, 11, 29, 0, 12, 41, 14, 34, 48, 5, 63, 27, 0, 2, 16, 0, 100, 1, 1, 11, 0, 0, 26, 57, 0, 6, 68, 0, 21, 0, 0, 20, 89, 4, 5, 44, 24, 4, 3, 4, 0, 6, 13, 100, 0, 0, 8, 9, 0, 45, 2, 19, 2, 41, 95, 17, 81, 13, 0, 18, 29, 12, 3, 13, 72, 0, 8, 13, 3, 1, 56, 87, 12, 37, 4, 15, 44, 28, 0, 14, 0, 19, 1, 18, 29, 0, 29, 40, 8]

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = sns.distplot(step_counts_100_1000)
_ = ax.set_title("Histogram of Steps Taken During Random Walks - Max-Steps of 100 w/ 1,000 trials")
_ = ax.set_ylabel("% of data")
_ = ax.set_xlabel("# of steps taken")

## What were the most common nodes seen?

In [ ]:
for k, v in random_walks[0].nodes_seen.items():
    print(k, v)

In [ ]:
all_nodes_seen = defaultdict(int)
for walk in random_walks:
    for node, seen_count in walk.nodes_seen.items():
        all_nodes_seen[node] += seen_count

In [ ]:
len(all_nodes_seen)

In [ ]:
all_nodes_seen_counter = Counter(all_nodes_seen)

### Most common nodes seen on walks...

In [ ]:
all_nodes_seen_counter.most_common()[:50]

### In contrast, the most common nodes in the dataset by degree...

In [ ]:
Counter(dict(G_combined.degree)).most_common()[:50]

### And the most common nodes in the graph by in-degree...

In [ ]:
Counter(dict(G_combined.in_degree)).most_common()[:50]

In [ ]:
top_50_seen_in_walks = dict(all_nodes_seen_counter.most_common()[:50])
top_50_seen_in_walks_set = set(x for x in top_50_seen_in_walks)

top_50_by_degree = dict(Counter(dict(G_combined.degree)).most_common()[:50])
top_50_by_degree_set = set(x for x in top_50_by_degree)

top_50_by_in_degree = dict(Counter(dict(G_combined.in_degree)).most_common()[:50])
top_50_by_in_degree_set = set(x for x in top_50_by_in_degree)

## Seen in random walks, but not in the top number as far as in-degree

In [ ]:
walks_but_not_in_degree = sorted([(x, top_50_seen_in_walks[x]) for x in top_50_seen_in_walks_set - top_50_by_in_degree_set], key=lambda x: x[1], reverse=True)
walks_but_not_in_degree

## Seen in top in-degree, but not seen as much in random walks

In [ ]:
in_degree_but_not_walks = sorted([(x, top_50_by_in_degree[x]) for x in top_50_by_in_degree_set - top_50_seen_in_walks_set], key=lambda x: x[1], reverse=True)
in_degree_but_not_walks

# Why would we see something in a walk if it doesn't have one of the highest in-degrees?
Let's try to compare the nodes that were seen in walks but weren't in the top in-degree, vs the ones that were in the top in-degree but weren't seen in walks...

In [ ]:
degree_centrality = nx.centrality.degree_centrality(G_combined)

In [ ]:
walks_not_degree_centrality = [degree_centrality[node[0]] for node in walks_but_not_in_degree]
walks_not_degree_centrality

In [ ]:
degree_not_walks_centrality = [degree_centrality[node[0]] for node in in_degree_but_not_walks]
degree_not_walks_centrality

In [ ]:
print(np.mean(walks_not_degree_centrality), "-", np.mean(degree_not_walks_centrality))

In [ ]:
# PageRank isn't implemented for Multigraph, so I'm going to use the weighted graph instead
pr_combined = nx.pagerank(G_weighted)

In [ ]:
[pr_combined[node[0]] for node in walks_but_not_in_degree]

In [ ]:
[pr_combined[node[0]] for node in in_degree_but_not_walks]

## By all measures, it looks like we shouldn't have seen the random_walk_but_not_in_degree nodes... Let's take a look at some of them.

In [ ]:
print(G_combined["thebeachboys"])

In [ ]:
print(G_combined["beachboys"])

Ahhhhhh...you know what, I wonder if these are some of the 2-cycles that we found. We randomly got placed in them and then just went back and forth...

In [ ]:
walk = RandomWalk(G_weighted, current_node="beachboys")
for i in range(1000):
    walk.walk()

In [ ]:
walk.nodes_seen

That seems to explain that.

But what about the ones that have over 1000? We only took 1000 steps, so something else has to be going on there.

```
[('emeraldcoastbeer', 1976),
 ('pensacolabeer', 1975),
 ('mpos', 1497),
 ('modelwestdistrict', 1496),
 ('allgamersunited', 1495),
 ('judgedvideogames', 1495),
 ('bloomingtonnormal', 1490),
 ('blono', 1489),
 ```

In [ ]:
print(G_combined['emeraldcoastbeer'])

In [ ]:
print(G_combined['pensacolabeer'])

Perhaps we randomly selected it twice! I think that's plausible. What about the ones that are in the middle, ~1400?

In [ ]:
print(G_combined['mpos'])

In [ ]:
print(G_combined['modelwestdistrict'])

In [ ]:
print(G_combined['allgamersunited'])

In [ ]:
print(G_combined['judgedvideogames'])

Hmm. Honestly not sure what's going on there.